In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135824")
exp = Experiment(workspace=ws, name="quick-starts-ws-135824")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FVATXGVLS to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-135824
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-135824


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "hyperdriveml"
cpu_cluster = ComputeTarget(ws,cpu_cluster_name)

try:
       cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
       print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
%%writefile conda_dependencies.yml

dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pip:
    - azureml-defaults

Writing conda_dependencies.yml


In [4]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({"max_iter" : choice(100,150,200,250),
                                "C" : choice(0.01,0.05,0.2,1,5,10)
                                })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target=cpu_cluster,                      
                      environment=sklearn_env)


# est = SKLearn(source_directory='./',               
#             compute_target=cpu_cluster,
#             entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#experimnt = Experiment(ws,exp)
hd_run = exp.submit(hd_config)
hd_run.wait_for_completion(show_output=True)
RunDetails(hd_run).show()
### YOUR CODE HERE ###

RunId: HD_9f812997-0631-461f-978b-844a8a428b1f
Web View: https://ml.azure.com/experiments/quick-starts-ws-135824/runs/HD_9f812997-0631-461f-978b-844a8a428b1f?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-135824/workspaces/quick-starts-ws-135824

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-23T20:49:35.463927][API][INFO]Experiment created<END>\n""<START>[2021-01-23T20:49:36.469092][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-23T20:49:36.5967650Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-23T20:49:36.130582][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_9f812997-0631-461f-978b-844a8a428b1f
Web View: https://ml.azure.com/experiments/quick-starts-ws-135824/runs/HD_9f812997-0631-461f-978b-844a8a428b1f?wsid=/subsc

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-135824,HD_9f812997-0631-461f-978b-844a8a428b1f_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
print("Best run details", best_run.get_details())
print('\t')
print("Best run file names:", best_run.get_file_names())
model = best_run.register_model(model_name='bankmark_hyperdrive_best_run', model_path='./outputs/model.joblib')
print(model.name, model.id,model.version,sep='\t')

Best run details {'runId': 'HD_9f812997-0631-461f-978b-844a8a428b1f_1', 'target': 'hyperdriveml', 'status': 'Completed', 'startTimeUtc': '2021-01-23T20:55:26.092059Z', 'endTimeUtc': '2021-01-23T20:58:12.499814Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '2272f6f3-2b5a-4023-995c-e0dea799876f', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.01', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'hyperdriveml', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'priority': None, 'credentialPassthrough': False, 'environment': {'name': 'sklearn-env', 'version': 'Autosave_2021-01-23T20:49:35Z_29860885', 'python': {'i

In [9]:
hd_run.get_metrics()

{'HD_9f812997-0631-461f-978b-844a8a428b1f_0': {'Regularization Strength:': 0.05,
  'Max iterations:': 250,
  'Accuracy': 0.9072837632776934},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_1': {'Regularization Strength:': 0.01,
  'Max iterations:': 100,
  'Accuracy': 0.9088012139605463},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_10': {'Regularization Strength:': 5.0,
  'Max iterations:': 150,
  'Accuracy': 0.9074355083459787},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_11': {'Regularization Strength:': 0.05,
  'Max iterations:': 150,
  'Accuracy': 0.9080424886191198},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_12': {'Regularization Strength:': 1.0,
  'Max iterations:': 150,
  'Accuracy': 0.9077389984825494},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_13': {'Regularization Strength:': 10.0,
  'Max iterations:': 100,
  'Accuracy': 0.908649468892261},
 'HD_9f812997-0631-461f-978b-844a8a428b1f_14': {'Regularization Strength:': 0.2,
  'Max iterations:': 200,
  'Accuracy': 0.9074355083459787},
 'HD_

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.core.dataset import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=data_url)


In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y
label = "y"

In [12]:
x.head()
x.to_csv("bankmarketing_train.csv")

In [13]:
from azureml.core.dataset import Dataset
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['bankmarketing_train.csv'],
                        target_path = './',
                        overwrite=True,
                        show_progress=True)
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, "./bankmarketing_train.csv")])

Uploading an estimated of 1 files
Uploading bankmarketing_train.csv
Uploaded bankmarketing_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name=label,
    n_cross_validations=3)

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
#ws = Workspace.from_config()
#experiment = Experiment(ws, "exp")
run = exp.submit(automl_config, show_output=True)
run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_375d101e-8fad-47e3-810a-6a97847f543b

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

{'runId': 'AutoML_375d101e-8fad-47e3-810a-6a97847f543b',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-23T21:08:31.396023Z',
 'endTimeUtc': '2021-01-23T21:41:58.928154Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"quick-starts-ws-135824","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group":"aml-quickstarts-135824","workspace_name":"quick-starts-ws-135824","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classe

In [16]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: quick-starts-ws-135824,
Id: AutoML_375d101e-8fad-47e3-810a-6a97847f543b_34,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                intercept_scaling=1,
                                                                                                l1_ratio=None,
                                         

In [17]:
from azureml.interpret import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(best_run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [18]:
client = ExplanationClient.from_run_id(ws, 'quick-starts-ws-135744', best_run.id)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [19]:
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [20]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [0.5383595320446376, 0.3912522512445099, 0.16865529451041963, 0.08177701960531822]
global importance names: ['duration', 'nr.employed', 'cons.conf.idx', 'emp.var.rate']


In [21]:
model_name = best_run.properties['model_name']

script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [22]:
cpu_cluster.delete()